### Importování dat

In [1]:
import pandas as pd

df = pd.read_csv('psp_records_list.csv')
df.tail()

,Line
19816,"pan,poslanec,mikuláš,ferjenčík,faktickou,pozná..."
19817,"já,se,také,pokusím,vystoupit,naposledy,každopá..."
19818,"pan,předseda,kalousek"
19819,"rámci,konsenzuální,diskuse,navrhuji,kompromis,..."
19820,"tuto,chvíli,nemám,nikoho,přihlášeného,do,obecn..."


### Zbavení se diakritky a změnění písmen na malé

In [2]:
def lower_text(df):
    for index in range(df.size):
        df.values[index][0] = df.values[index][0].lower()
        
df_lower = lower_text(df)

In [3]:
import unicodedata

def deaccent(unistr):
    return "".join(aChar 
                   for aChar in unicodedata.normalize("NFD", unistr) 
                   if not unicodedata.combining(aChar))

for i in range(df.size):
    df.iloc[i,0] = deaccent(df.iloc[i,0])

### Listy vět sestavené z jednotlivých slov

In [4]:
def get_sentences(df):
    df_values = df.values.tolist()
    all_sentences = []
    for index in range(df.size):
        all_sentences.append(df_values[index][0].split(',')) 
    return all_sentences

all_sentences = get_sentences(df)

### Nalezení a zbavení se nejfrekventovanějších slov, nemající velký význam

In [5]:
from nltk.probability import FreqDist

def frequent(lst):
    fdist = FreqDist()
    for i in range(len(lst)):
        for j in range(len(lst[i])):
            fdist[lst[i][j]] += 1
    return(fdist)
            
frequency_bef = frequent(all_sentences)

In [6]:
most_common = []
for i in range(41):
    most_common.append(frequency_bef.most_common()[i][0])

In [8]:
def remove_common_words(all_sentences):
    for word in most_common:
        for index in range(df.size):
            while word in all_sentences[index]:
                all_sentences[index].remove(word)
                
remove_common_words(all_sentences)

### Odstranění tzv. stopwords

In [9]:
stopwords = pd.read_json("stop_words_czech.json")

for i in range(stopwords.size):
    stopwords.iloc[i,0] = deaccent(stopwords.iloc[i,0])
    
for i in range(stopwords.size):
    most_common.append(stopwords.iloc[i,0])

In [10]:
remove_common_words(all_sentences)

In [11]:
frequency_aft = frequent(all_sentences)

In [12]:
x=0
y=0

for value in frequency_bef.values():
    x += value
for value in frequency_aft.values():
    y += value

print("\nNumber of characters in all_sentences:",x,"\nNumber of characters after removing most_common:",y,
      "\nDifference:",x-y)


Number of characters in all_sentences: 2897666 
Number of characters after removing most_common: 1737029 
Difference: 1160637


# Word2Vec - knihovna gensim

In [13]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_sentences, size=200, min_count=5, window=10)

In [14]:
vocabulary = word2vec.wv.vocab

### Slovo reprezentováno jako vektor

In [15]:
word2vec.wv['zeman']

array([ 9.99518670e-03, -1.56774163e-01, -2.10363537e-01,  1.78140938e-01,
        6.12396002e-02, -8.18592533e-02,  2.75070239e-02, -1.41844869e-01,
        8.11821520e-02,  4.08635363e-02, -1.84148237e-01, -1.72700316e-01,
       -2.52995968e-01,  6.01946227e-02, -1.39390260e-01,  1.20326513e-02,
       -7.71925077e-02, -2.00754210e-01,  2.27251083e-01,  6.18844815e-02,
       -1.62890881e-01, -9.68493298e-02, -2.46805191e-01, -2.33188152e-01,
        2.65404224e-01,  4.17446457e-02, -1.45204276e-01, -1.21777773e-01,
        1.71395466e-01,  3.59734930e-02, -4.85502295e-02,  5.34483939e-02,
        1.56316474e-01, -1.10934846e-01, -1.12393945e-01, -1.46192700e-01,
        1.48919448e-01,  7.05435500e-02,  8.10600258e-03,  1.08697675e-01,
        4.78863195e-02, -1.46531284e-01,  7.37181902e-02, -1.32097146e-02,
       -3.01055163e-02,  3.67280073e-03,  1.14617996e-01,  1.90832481e-01,
        1.30836636e-01, -8.33729804e-02,  1.59014702e-01, -7.93044120e-02,
        1.33178830e-01,  

### Nalezení n nejpodobnějších slov

In [16]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> Miloš


[('zeman', 0.9426842331886292),
 ('bohuslav', 0.9029508233070374),
 ('napsal', 0.8974103927612305),
 ('masaryk', 0.8768518567085266),
 ('prohlasil', 0.867906928062439),
 ('primator', 0.8656466007232666),
 ('stal', 0.8569535613059998),
 ('miroslav', 0.8561363816261292),
 ('podepsal', 0.8503516912460327),
 ('tvrdil', 0.8493710160255432)]

In [17]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> Zeman


[('milos', 0.9426842331886292),
 ('miroslav', 0.909707248210907),
 ('masaryk', 0.892728328704834),
 ('tvrdil', 0.8904792070388794),
 ('bohuslav', 0.887295126914978),
 ('sobotka', 0.8868386745452881),
 ('garrigue', 0.8834864497184753),
 ('podepsal', 0.8725541830062866),
 ('stal', 0.8670616149902344),
 ('exministr', 0.8664671182632446)]

In [18]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> TOP


[('ods', 0.9571278691291809),
 ('poslaneckeho', 0.9429056644439697),
 ('stan', 0.9375035166740417),
 ('piratu', 0.9292595386505127),
 ('kscm', 0.9281211495399475),
 ('klubu', 0.9237409830093384),
 ('klub', 0.9017151594161987),
 ('jmenem', 0.8877667784690857),
 ('poslaneckych', 0.869156539440155),
 ('cssd', 0.8651964068412781)]

In [19]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> ANO


[('cssd', 0.7866946458816528),
 ('hnuti', 0.7783950567245483),
 ('komunistu', 0.7622459530830383),
 ('jednotni', 0.7269113659858704),
 ('kscm', 0.7230626940727234),
 ('asocialni', 0.7130089998245239),
 ('pravda', 0.7061961889266968),
 ('ods', 0.6985846757888794),
 ('lavic', 0.6962398886680603),
 ('prednaset', 0.6937940120697021)]